In [1]:
import Python
import Dispatch
let pd = Python.import("pandas")
let np = Python.import("numpy")

# Library

## Utils

In [2]:
extension String {
    func strip() -> Substring {
        return self.drop{$0==" "}
    }
}

// Time how long it takes to run the specified function, optionally taking
// the average across a number of repetitions.
public func time(repeating: Int = 1, _ function: () -> ()) {
    guard repeating > 0 else { return }
    
    // Warmup
    if repeating>1 {function()}
    
    var times = [Double]()
    for _ in 1...repeating {
        let start = DispatchTime.now()
        function()
        let end = DispatchTime.now()
        let nanoseconds = Double(end.uptimeNanoseconds - start.uptimeNanoseconds)
        let milliseconds = nanoseconds / 1e6
        times.append(milliseconds)
    }
    print("average: \(times.reduce(0.0, +)/Double(times.count)) ms,   " +
          "min: \(times.reduce(times[0], min)) ms,   " +
          "max: \(times.reduce(times[0], max)) ms")
}


## Common types

## DataStorage

In [10]:
let a:Vector<Float> = [1.0, 2, 4]

: 

In [3]:
var x: [Float] = [1,2,3]
x

▿ 3 elements
  - 0 : 1.0
  - 1 : 2.0
  - 2 : 3.0


In [4]:
x.asum

: 

In [4]:
public protocol DataStorage {
    associatedtype Data
    var data: Data { get set }
    var dims: Int { get }
    static func new(random d: Int, lowerLimit l: Float, upperLimit u: Float) -> Self
    func sum() -> Float
}

### Plain swift implementation

In [5]:
public struct NativeDataStorage: DataStorage{
    public var data: [Float]
    
    public var dims: Int {
        return data.count
    }
    
    static public func new(random d: Int, lowerLimit l: Float, upperLimit u: Float) -> NativeDataStorage {
        return NativeDataStorage(data: (0..<d).map{ _ in Float.random(in: l...u) })
    }
    
    public func sum() -> Float {
        return data.reduce(0, +)
    }
}

### TensorFlow implementation

In [6]:
public struct TensorDataStorage: DataStorage {
    public var data: TF
    
    public var dims: Int {
        return data.shape[0]
    }
    
    static public func new(random d: Int, lowerLimit l: Float, upperLimit u: Float) -> TensorDataStorage {
        return TensorDataStorage(data: TF(randomUniform: [d]) * (u-l) + l)
    }
    
    public func sum() -> Float {
        return Float(data.sum())!
    }
    
}

### BaseMath implementation

In [7]:
// TODO

### SwiftyMKL implementation

In [8]:
// TODO

## ContraintParams

In [9]:
public protocol ConstraintParams {
    associatedtype Data
    var data: [Data] { get set }
    var maxTime: Int { get }
}

### Plain swift implementation

In [10]:
public struct NativeConstraintParams: ConstraintParams {
    public var data: [[Float]]
    
    public var maxTime: Int {
        return data[0].count
    }
    
    public init(data: [[Float]]) {
        assert(Set(data.map{ $0.count }).count == 1) // All params have same lenght
        self.data = data
    }
    
    public init(data: [Float]) {
        self.init(data: [data])
    }
}

In [11]:
var x = NativeConstraintParams(data: [1,2,3])
x.maxTime

3


In [12]:
var x = NativeConstraintParams(data: [[1,2,3],[4,5,3]])
x.maxTime

3


### TensorFlow implementation

In [13]:
public struct TensorConstraintParams: ConstraintParams {
    public var data: [TF]
    
    public var maxTime: Int {
        return data[0].shape[0]
    }
    
    public init(data: [TF]) {
        assert(data.allSatisfy{ $0.rank == 1 }) // All have rank 1
        assert(Set(data.map{ $0.shape[0] }).count == 1) // All params have same lenght
        self.data = data            
    }
    
    public init(data: TF) {
        self.init(data: [data])
    }
    
    public init(data: [Float]) {
        self.init(data: TF(data))
    }
}

In [14]:
var x = TensorConstraintParams(data: TF([1,2,3]))
x.maxTime

3


In [15]:
var x = TensorConstraintParams(data: [TF([1,2,3]),TF([2,3,4])])
x.maxTime

3


### BaseMath implementation

In [16]:
// TODO

### SwiftyMKL implementation

In [17]:
// TODO

## Individual

In [18]:
public struct Individual<DS: DataStorage> {
    public typealias Indiv=Individual<DS>
    public typealias Indivs=[Indiv]
    
    public var storage: DS {
        didSet {
            isFeasible = false
            fitnessValue = nil
            constraints = nil
        }
    }
    public var fitnessValue: Float?=nil
    public var constraints: DS? = nil {
        didSet { 
            if let c = constraints { constraintsSum = c.sum() }
        }
    }
    public var constraintsSum: Float? = nil
    public var isFeasible: Bool = false
    public let lowerLimit: Float
    public let upperLimit: Float
    
    public init(storage: DS, lowerLimit: Float, upperLimit: Float) {
        self.storage = storage
        self.lowerLimit = lowerLimit
        self.upperLimit = upperLimit
    }
    
    public init(random d: Int, lowerLimit l: Float = -5, upperLimit u: Float = 5) {
        self.init(storage: DS.new(random: d, lowerLimit: l, upperLimit: u), lowerLimit: l, upperLimit: u)
    }   
}

In [19]:
Individual<TensorDataStorage>(random: 5).storage.data

[ 2.0134687,  4.2453203, -4.2955008,   4.522361,  1.8680453]


In [20]:
Individual<NativeDataStorage>(random: 5).storage.data

▿ 5 elements
  - 0 : 0.5130062
  - 1 : -4.981606
  - 2 : -2.891615
  - 3 : 2.2364187
  - 4 : 3.0722132


## Population

In [21]:
public struct Population<DS: DataStorage> {
    public typealias Indiv=Individual<DS>
    public typealias Indivs=[Indiv]
    public typealias Pop=Population<DS>
    
    public var individuals: Indivs
    
    public init(individuals: Indivs) {
        self.individuals = individuals
    }

    public init(random n: Int, dimensions d: Int, lowerLimit l: Float = -5, upperLimit u: Float = 5) {
        var res: Indivs = []
        for _ in 0..<n { res.append(Indiv(random: d, lowerLimit: l, upperLimit: u)) }        
        self.init(individuals: res)
    }
}

In [22]:
var x = Population<TensorDataStorage>(random: 2, dimensions: 5)
x.individuals.map{ $0.storage.data }

▿ 2 elements
  - 0 : [-3.9655697, -2.1292508,  2.1890879, -4.6493616, 0.51372623]
  - 1 : [  0.9031143,  -3.0808496,   2.5267959, -0.90955734,  -1.5661442]


In [23]:
var x = Population<NativeDataStorage>(random: 2, dimensions: 3)
x.individuals.map{ $0.storage.data }

▿ 2 elements
  ▿ 0 : 3 elements
    - 0 : 4.741678
    - 1 : 1.0449672
    - 2 : 0.10813999
  ▿ 1 : 3 elements
    - 0 : 2.4675188
    - 1 : 3.4346504
    - 2 : 0.9351883


In [24]:
time(repeating: 10, { var x = Population<NativeDataStorage>(random: 100, dimensions: 1000) } )

average: 61.237189799999996 ms,   min: 45.680111 ms,   max: 195.171014 ms


In [25]:
time(repeating: 10, { var x = Population<TensorDataStorage>(random: 100, dimensions: 1000) } )

average: 13.4846283 ms,   min: 11.827623 ms,   max: 15.304748 ms


## Optimization

### Error handling

In [26]:
public enum OptAction: Error {
    case stopEvolve
    case stopIndividual
    case stopGeneration
    case stopRun
}

### Evolution mechanism

In [27]:
extension Population {
    public func pick(but: Int) -> [DS.Data] {
        var picked: [DS.Data] = []
        while picked.count < 3 {
            let i = Int.random(in: 0..<individuals.count)
            if i != but { picked.append(individuals[i].storage.data) }
        }
        return picked
    }
}

In [28]:
extension Individual where DS==NativeDataStorage {
    public func evolve(picked: [[Float]], CR: Float, betaMin: Float, betaMax: Float) -> [Float] {
        var res: [Float] = storage.data
        var dims: [Int] = []
        dims.append(Int.random(in: 0..<storage.dims))
        (0..<storage.dims).enumerated().filter{ _ in Float.random(in: 0...1)<CR }.map{
            dims.append($1)
        }
        let selectedDims = Set(dims)
        let factors: [Float] = (0..<selectedDims.count).map{ _ in Float.random(in: betaMin...betaMax) }
        for (d,f) in zip(selectedDims, factors) {
            res[d] = max(min(picked[0][d] + f*picked[1][d] - picked[2][d], lowerLimit), upperLimit)
        }
        return res
    }
}

In [30]:
public struct EvolveMechanism<DS: DataStorage> {
    public typealias Indiv=Individual<DS>
    public typealias Indivs=[Indiv]
    public typealias Pop=Population<DS>
    public var n: Int = 3
    public var CR: Float = 0.3
    public var betaMin: Float = 0.2
    public var betaMax: Float = 0.8
    
    func call(_ population: Pop, _ idx: Int) -> Indiv {
        var newIndividual: Indiv = population.individuals[idx]
        var picked: [DS.Data] = population.pick(but: idx)
        newIndividual.storage.data = newIndividual.evolve(picked: picked, CR: CR, betaMin: betaMin, betaMax: betaMax)
        return newIndividual
    }
}

: 

### Main class

In [49]:
final public class Optimization<DS: DataStorage, CP: ConstraintParams> where DS.Data==CP.Data {
    // Common types
    public typealias Indiv=Individual<DS>
    public typealias Indivs=[Individual<DS>]
    public typealias Pop=Population<DS>
    public typealias FitnessFn=((Indiv) -> Float)
    public typealias ConstraintFun=((Indiv,Float) -> Float)
    
    // Main properties
    public var population: Pop
    public var getFitness: FitnessFn
    public var getConstraints: ConstraintFun
    public let constraintParams: CP
    public let frequency: Int
    public let maxTime: Int
    public let maxEvals: Int? = nil
    
    // Editable functions
    public var evolveMechanism: EvolveMechanism<DS> = EvolveMechanism()
    
    // Main entities on the loop
    public var currentIndividual: Indiv? = nil
    public var currentIndividualBkup: Indiv? = nil
    public var currentIndividualIdx: Int? = nil
    // Record properties
    public private(set) var currentGeneration: Int = .zero
    public private(set) var currentEvaluation: Int = .zero
    public private(set) var currentTime: Int = .zero
    public private(set) var currentBest: Indiv? = nil
    
    open class Delegate {
        open var order: Int { return 0 }
        public init () {}
        
        open func onRunBegin(opt: Optimization) throws {}
        open func onGenerationBegin(opt: Optimization) throws {}
        open func onIndividualBegin(opt: Optimization) throws {}
        open func onEvolveBegin(opt: Optimization) throws {}
        open func onEvolveEnd(opt: Optimization) throws {}
        open func onIndividualEnd(opt: Optimization) throws {}
        open func onGenerationEnd(opt: Optimization) throws {}
        open func onRunEnd(opt: Optimization) throws {}
    }

    public var delegates: [Delegate] = [] {
        didSet { delegates.sort { $0.order < $1.order } }
    }
    
    public func addDelegate(_ delegate: Optimization.Delegate) {
        delegates.append(delegate)
    }
    
    public func addDelegates(_ delegates: [Optimization.Delegate]) {
        self.delegates += delegates
    }
    
    // Initialization
    public init(population: Pop, getFitness: @escaping FitnessFn, getConstraints: @escaping ConstraintFun,
                constraintParams: CP, frequency: Int) {
        self.population = population
        self.getFitness = getFitness
        self.getConstraints = getConstraints
        self.constraintParams = constraintParams
        self.frequency = frequency
        self.maxTime = constraintParams.maxTime
    }
}

### Run logic

In [50]:
extension Optimization {
    public func run(_ generations: Int) throws {
        do{
            try delegates.forEach { try $0.onRunBegin(opt: self) }
            for i in 0..<generations {
                try runGeneration(i)
            }
        } catch OptAction.stopRun {}
        try delegates.forEach { try $0.onRunEnd(opt: self) }
    }
    
    public func runGeneration(_ i: Int) throws {
        do{
            currentGeneration += 1
            try delegates.forEach { try $0.onGenerationBegin(opt: self) }
            try population.individuals.enumerated().forEach { try processIndividual($0,$1) }
            try delegates.forEach { try $0.onGenerationEnd(opt: self) }
        } catch OptAction.stopGeneration {}
    }
    
    public func processIndividual(_ i: Int, _ individual: Indiv) throws {
        do{
            try delegates.forEach { try $0.onIndividualBegin(opt: self) }
            currentIndividual = individual
            currentIndividualBkup = individual
            currentIndividualIdx = i
            try evolve()
//             eval_fitness(x)
//             eval_constraints(x)
        } catch OptAction.stopIndividual {}
        try delegates.forEach { try $0.onIndividualEnd(opt: self) }
    }
    
    public func evolve() throws {
        do{
            try delegates.forEach { try $0.onEvolveBegin(opt: self) }
            currentIndividual = evolveMechanism(population, currentIndividualIdx!)
            population.individuals[currentIndividualIdx!] = currentIndividual!
        } catch OptAction.stopEvolve {}
        try delegates.forEach { try $0.onEvolveEnd(opt: self) }
    }
    
}

### Delegates

#### Recorder

In [51]:
extension Optimization {
    public class RecorderDelegate: Delegate {
        public override func onRunBegin(opt: Optimization) {
            print("Starting run...")
        }
        
        public override func onGenerationBegin(opt: Optimization) {
            print("Generation \(opt.currentGeneration)...")
        }
        
        public override func onRunEnd(opt: Optimization) {
            print("Ending run...")
        }
    }
    
    public func makeRecorderDelegate() -> RecorderDelegate { return RecorderDelegate() }
}

# Use

In [63]:
var df = pd.read_csv("/home/renato/github/DENN/data/medium/dC_01.csv")
let abSource = df.columns.map{Float(String($0)!.strip())!}
let abNative = NativeConstraintParams(data: abSource)
let abTF = TensorConstraintParams(data: abSource)

In [64]:
let D = 30
let frequency = 1_000
let totalGenerations = abNative.maxTime * frequency + 1000

In [65]:
func getFitness(_ indv: Individual<TensorDataStorage>) -> Float {
    return Float(pow(indv.storage.data, 2).sum())!
}

func getConstraint(_ indiv: Individual<TensorDataStorage>, _ b: Float) -> Float {
    return Float((sqrt(Float(D)) * indiv.storage.data).sum() - b)!
}

In [66]:
var population = Population<TensorDataStorage>(random: 20, dimensions: D)

In [83]:
var opt = Optimization(population: population, getFitness: getFitness, getConstraints: getConstraint,
                       constraintParams: abTF, frequency: frequency)

In [84]:
opt.addDelegate(opt.makeRecorderDelegate())

In [96]:
opt.run(5)

Starting run...
Generation 56...
Generation 57...
Generation 58...
Generation 59...
Generation 60...
Ending run...


In [97]:
opt.currentGeneration

60


# ----